# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import csv

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
weather_data_to_load = "../WeatherPy/weather_df.csv"
weather_data = pd.read_csv(weather_data_to_load)
dropna_weather_data = weather_data.dropna()


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = dropna_weather_data[["Latitude", "Longitude"]]

humidity = dropna_weather_data["Humidity (%)"].astype(float)

In [6]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [7]:
# Filter vacation with zero cloudiness
vacation_no_cloud = dropna_weather_data[dropna_weather_data["Cloudiness (%)"] == 0]
# Filter vacation with max temp above 70 degrees F
vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp (F)"] > 70]
# Filter vacation with max temp below 80 degrees F
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temp (F)"] < 80]
# Filter vacation with wind speed below 10 mph
vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed (mph)"] < 10]
# Filter vacation with humidity below 60 %
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity (%)"] < 60]
# Set Index
hotel_df = perfect_vacation.reset_index()
hotel_df

,index,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,312,tete,MZ,2020-10-19 21:16:03,-16.16,33.59,0.0,57.0,75.092,2.59
1,373,barentu,ER,2020-10-19 21:16:11,15.11,37.59,0.0,44.0,71.366,1.38
2,414,karratha,AU,2020-10-19 21:16:17,-20.74,116.85,0.0,41.0,79.772,2.48
3,465,bilma,NE,2020-10-19 21:16:24,18.69,12.92,0.0,17.0,78.260,1.65
4,537,tessalit,ML,2020-10-19 21:16:33,20.20,1.01,0.0,17.0,79.916,3.43


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [8]:
vaca_locations = hotel_df[["Latitude", "Longitude"]]

vaca_humidity = hotel_df["Humidity (%)"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))

In [9]:

# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(hotel_df["City"])):

    lat = hotel_df.loc[city]["Latitude"]
    lng = hotel_df.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
hotel_df["Nearest Hotel"] = hotels
hotel_df

,index,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Nearest Hotel
0,312,tete,MZ,2020-10-19 21:16:03,-16.16,33.59,0.0,57.0,75.092,2.59,Zambeze Paraíso Misterioso
1,373,barentu,ER,2020-10-19 21:16:11,15.11,37.59,0.0,44.0,71.366,1.38,Nearest hotel not found
2,414,karratha,AU,2020-10-19 21:16:17,-20.74,116.85,0.0,41.0,79.772,2.48,ibis Styles Karratha
3,465,bilma,NE,2020-10-19 21:16:24,18.69,12.92,0.0,17.0,78.260,1.65,Expeditions Ténére voyages
4,537,tessalit,ML,2020-10-19 21:16:33,20.20,1.01,0.0,17.0,79.916,3.43,Nearest hotel not found


In [10]:
# NOTE: Do not change any of the code in this cell


# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Nearest Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [11]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vaca_fig.add_layer(markers)

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))